##### Copyright 2022 The TensorFlow GNN Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, eicther express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# An in-depth look at TF-GNN for OGBN-MAG


<table class="tfo-notebook-buttons" align="left">
  <td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/gnn/blob/master/examples/notebooks/ogbn_mag_indepth.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/gnn/blob/main/examples/notebooks/ogbn_mag_indepth.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
</table>

### Abstract

This tutorial solves the same task as the basic "[OGBN-MAG end-to-end](https://colab.research.google.com/github/tensorflow/gnn/blob/master/examples/notebooks/ogbn_mag_e2e.ipynb)" tutorial, but at a lower API level: it introduces to you to the `tfgnn.GraphTensor` type, uses it to build a Graph Neural Network in Keras, and demonstrates how to train the network without the abstractions provided by the TF-GNN Runner.

This tutorial is intended for advanced users who have a working knowledge of TF2/Keras already and need to exercise greater control over the model definition and the training code.

## Colab set-up

In [ ]:
!pip install -q tensorflow-gnn || echo "Ignoring package errors..."

In [ ]:
import functools
import itertools
import os
import random
import re

from google.protobuf import text_format
os.environ["TF_USE_LEGACY_KERAS"] = "1"  # Select tf.keras version 2, not 3.
import tensorflow as tf
import tensorflow_gnn as tfgnn

print(f"Running TF-GNN {tfgnn.__version__} under TensorFlow {tf.__version__}.")

Running TF-GNN 0.5.0 under TensorFlow 2.9.2.


## Introduction

### Problem statement and dataset

OGBN-MAG is [Open Graph Benchmark](https://ogb.stanford.edu)'s Node classification task on a subset of the [Microsoft Academic Graph](https://www.microsoft.com/en-us/research/publication/microsoft-academic-graph-when-experts-are-not-enough/). The [basic tutorial](https://colab.research.google.com/github/tensorflow/gnn/blob/master/examples/notebooks/ogbn_mag_e2e.ipynb) has explained the dataset and the task in detail. To recap, the OGBN-MAG dataset is one big graph with node sets "paper", "field_of_study", "author", and "institution" and edge sets "cites", "has_topic", "writes", and "affiliated_with", each connecting two particular node sets.

The task is to **predict the venue** (journal or conference, not represented in the graph itself) at which each of the papers has been published. Based on their "year" feature, the "paper" nodes are split into "train" (`year<=2017`), "validation" (`year==2018`), and "test"(`year==2019`).


### Approach

To stay scalable for even bigger datasets, we approach this task with **graph sampling**: Each "paper" node becomes one training example, expressed by a subgraph that has the node to be classified as its root and stores a sample of its neighborhood in the original graph. The sample is taken by going out a fixed number of steps along specific edge sets, and randomly downsampling the edges in each step if they are too numerous.

The actual **TensorFlow model** runs on batches of these sampled subgraphs, applies a Graph Neural Network to propagate information from related nodes towards the root node of each batch, and then applies a softmax classifier to predict one of 349 classes (each venue is a class).


## Data preparation and graph sampling


Data preparation and sampling are exactly identical to the [basic tutorial](https://colab.research.google.com/github/tensorflow/gnn/blob/master/examples/notebooks/ogbn_mag_e2e.ipynb) and not repeated here. To recap, the sampling expected by the model in this colab proceeds as follows:

  1. Start from all "paper" nodes.
  2. For each paper from 1, follow a random sample of "cites" edges to other "paper" nodes.
  3. For each paper from 1 or 2, follow a random sample of reversed "writes" edges to "author" nodes and store them as edge set "written".
  4. For each author from 3, follow a random sample of "writes" edges to more "paper" nodes.
  5. For each author from 3, follow a random sample of "affiliated_with" edges to "institution" nodes.
  6. For each paper from 1, 2 or 4, follow a random sample of "has_topic" edges to "field_of_study" nodes.


## Reading the sampled subgraphs

The result of sampling is available here (subject to this [license](https://storage.googleapis.com/download.tensorflow.org/data/ogbn-mag/sampled/v1/edge/LICENSE.txt)):


In [ ]:
input_file_pattern = "gs://download.tensorflow.org/data/ogbn-mag/sampled/v1/edge/samples-?????-of-00100"
graph_schema_file = "gs://download.tensorflow.org/data/ogbn-mag/sampled/v1/edge/schema.pbtxt"
graph_schema = tfgnn.read_schema(graph_schema_file)

TF-GNN's guide on [Describing your Graph](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/schema.md) explains what a graph schema is. Here is the one for this dataset of sampled subgraphs:

In [ ]:
graph_schema

context {
  features {
    key: "sample_id"
    value {
      dtype: DT_STRING
    }
  }
  features {
    key: "seed_id"
    value {
      dtype: DT_STRING
    }
  }
  metadata {
  }
}
node_sets {
  key: "author"
  value {
    features {
      key: "#id"
      value {
        dtype: DT_STRING
      }
    }
    metadata {
    }
  }
}
node_sets {
  key: "field_of_study"
  value {
    features {
      key: "#id"
      value {
        dtype: DT_STRING
      }
    }
    metadata {
    }
  }
}
node_sets {
  key: "institution"
  value {
    features {
      key: "#id"
      value {
        dtype: DT_STRING
      }
    }
    metadata {
    }
  }
}
node_sets {
  key: "paper"
  value {
    features {
      key: "#id"
      value {
        dtype: DT_STRING
      }
    }
    features {
      key: "feat"
      value {
        dtype: DT_FLOAT
        shape {
          dim {
            size: 128
          }
        }
      }
    }
    features {
      key: "labels"
      value {
        dtype: DT_IN

Training a neural network with stochastic gradient descent requires randomly shuffled training data, but ours is too big to fully reshuffle it on the fly while reading. Fortunately, the graph sampler tool has already reshuffled its outputs before writing to a sharded TFRecord file.

For speed, we want to read from several shards in parallel.
For distributed training, each trainer replica reads from its own subset of the shards. To achieve some randomization between training runs, each replica reshuffles the order of input shards and then shuffles examples within a moderate window.


In [ ]:
def _get_dataset(file_pattern, *, shuffle=False, filter_fn=None,
                 input_context=None):
  # For your own file system or GCS bucket, call the usual helper
  # filenames = tf.io.gfile.glob(file_pattern)
  # For gs://download.tensorflow.org, we avoid listing it and do
  filenames = _glob_sharded(file_pattern)
  ds = tf.data.Dataset.from_tensor_slices(filenames)
  if input_context and input_context.num_input_pipelines > 1:
    ds = ds.shard(input_context.num_input_pipelines,
                  input_context.input_pipeline_id)
  if shuffle:
    ds = ds.shuffle(len(filenames))

  def interleave_fn(filename):
    ds = tf.data.TFRecordDataset(filename)
    if filter_fn is not None:
      ds = ds.filter(filter_fn)
    return ds
  ds = ds.interleave(
      interleave_fn, cycle_length=10,
      deterministic=False, num_parallel_calls=tf.data.AUTOTUNE)
  if shuffle:
    ds = ds.shuffle(10000)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

def _glob_sharded(file_pattern):
  match = re.fullmatch(r"(.*)-\?\?\?\?\?-of-(\d\d\d\d\d)", file_pattern)
  if match is None:  # No shard suffix found.
    return [file_pattern]
  basename = match[1]
  n = int(match[2])
  return [f"{basename}-{i:05d}-of-{n:05d}" for i in range(n)]

Graph sampling stores each sampled subgraph in its output as one tf.Example proto, with structured feature names, as explained in the [Data Preparation](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/data_prep.md) guide. If you're curious, take a look here:

In [ ]:
demo_ds = _get_dataset(input_file_pattern.replace("-?????-of-00100", "-00000-of-00100"))

In [ ]:
def _example_to_text(example):
  """MessageToString() with fewer linebreaks."""
  lines = ["features {"]
  for k, v in sorted(example.features.feature.items()):
    v_str = text_format.MessageToString(v, as_one_line=True,
                                        use_short_repeated_primitives=True)
    lines.append("  feature {")
    lines.append(f"    key: \"{k}\"")
    lines.append(f"    value {{ {v_str} }}")
    lines.append("  }")
  lines.append("}")
  return "\n".join(lines)

for serialized_example in demo_ds.take(1):
  example = tf.train.Example.FromString(serialized_example.numpy())
  print(_example_to_text(example))

features {
  feature {
    key: "context/sample_id"
    value { bytes_list { value: "paper109537" } }
  }
  feature {
    key: "context/seed_id"
    value { bytes_list { value: "paper109537" } }
  }
  feature {
    key: "edges/affiliated_with.#size"
    value { int64_list { value: [6] } }
  }
  feature {
    key: "edges/affiliated_with.#source"
    value { int64_list { value: [0, 1, 2, 3, 4, 4] } }
  }
  feature {
    key: "edges/affiliated_with.#target"
    value { int64_list { value: [0, 0, 1, 0, 1, 2] } }
  }
  feature {
    key: "edges/cites.#size"
    value { int64_list { value: [1] } }
  }
  feature {
    key: "edges/cites.#source"
    value { int64_list { value: [0] } }
  }
  feature {
    key: "edges/cites.#target"
    value { int64_list { value: [20] } }
  }
  feature {
    key: "edges/has_topic.#size"
    value { int64_list { value: [307] } }
  }
  feature {
    key: "edges/has_topic.#source"
    value { int64_list { value: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 

The following helper lets us filter input data by OGB's specific rule for the test/validation/train split before parsing the full GraphTensor. (Users working on their own datasets may want to use separate datasets to begin with.)

In [ ]:
def _is_in_split(split_name):
  def filter_fn(serialized_example):
    features = {"years": tf.io.RaggedFeature(value_key="nodes/paper.year",
                                             dtype=tf.int64)}
    years = tf.io.parse_single_example(serialized_example, features)["years"]
    year = years[0]  # By convention, the root node is the first node.
    if split_name == "train":  # 629,571
      return year <= 2017
    elif split_name == "validation":  # 64,879
      return year == 2018
    elif split_name == "test":  # 41,939
      return year == 2019
    else:
      raise ValueError(f"Unknown split_name: '{split_name}'")
  return filter_fn

num_training_samples = 629571
num_validation_samples = 64879
num_test_samples = 41939

### The GraphTensor type

The cornerstone of model building with TF-GNN is the `tfgnn.GraphTensor` type. The following code cells demonstrate the essentials of using it. For more information, please see the comprehensive [Introduction to GraphTensor](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/graph_tensor.md).

`tfgnn.GraphTensor` is a TensorFlow Extension Type (or "composite tensor") that consists of multiple Tensors but can be used as one object in a tf.data.Dataset, and in the inputs/outputs of a Keras layer or a tf.function. Other examples of this are `tf.RaggedTensor` and `tf.SparseTensor`.

Every such type has a matching type spec (like a tf.Tensor has a tf.TensorSpec). A `GraphTensorSpec` can be created from the `GraphSchema` and contains information about ots node sets, their connection by edge sets, and the features on the graph. With that, we can parse `tf.Example`s as above into `GraphTensor` values:

In [ ]:
example_input_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)
parse = functools.partial(tfgnn.parse_single_example, example_input_spec)
graph = parse(serialized_example)

The `GraphTensor` is an immutable container of tensors, indexed by names. For example, here are the features and labels of all papers in this subgraph:

In [ ]:
graph.node_sets["paper"]["labels"]

<tf.Tensor: shape=(30, 1), dtype=int64, numpy=
array([[193],
       [265],
       [ 56],
       [193],
       [277],
       [277],
       [283],
       [ 83],
       [236],
       [193],
       [265],
       [193],
       [236],
       [144],
       [236],
       [ 97],
       [265],
       [265],
       [258],
       [258],
       [262],
       [258],
       [236],
       [236],
       [198],
       [145],
       [236],
       [265],
       [193],
       [277]])>

By convention, the root node of the sampled subgraph is stored as the first node (index 0) of its node set, so the target label for this subgraph is the first item in the tensor above.

A tf.data.Dataset of GraphTensors can freely be batched (and unbatched), which simply stacks (or unstacks) all the tensors that make up the individual graphs. Let's take edge set "cites" as an example.

In [ ]:
for batched_graph in demo_ds.map(parse).batch(3).take(1):
  print(batched_graph.edge_sets["cites"].adjacency.source)
  print(batched_graph.edge_sets["cites"].adjacency.target)

<tf.RaggedTensor [[0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]>
<tf.RaggedTensor [[20],
 [11, 90, 144, 167, 181, 196, 199, 201, 228, 264, 267, 287, 376, 401],
 [32, 88, 111, 175, 211, 235, 301, 309, 311, 350, 377, 521, 631, 688, 693,
  723, 760, 770, 788, 795, 940, 1100, 1127]                               ]>


The resulting RaggedTensors and per-graph indices are inconvenient for the modeling code. Hence GraphTensor allows to merge a batch of graphs into a single graph with contiguous indexing. The distinction between the inputs is preserved by the notion of **components** inside the merged graph: all node sets and edge sets keep track of the size of each component. (This is always true, there just happens to be a single component in each graph before merging.)

In [ ]:
merged_graph = batched_graph.merge_batch_to_components()
print(merged_graph.edge_sets["cites"].sizes)
print(merged_graph.edge_sets["cites"].adjacency.source)
print(merged_graph.edge_sets["cites"].adjacency.target)
print(merged_graph.node_sets["paper"].sizes)

tf.Tensor([ 1 14 23], shape=(3,), dtype=int32)
tf.Tensor(
[  0  30  30  30  30  30  30  30  30  30  30  30  30  30  30 503 503 503
 503 503 503 503 503 503 503 503 503 503 503 503 503 503 503 503 503 503
 503 503], shape=(38,), dtype=int32)
tf.Tensor(
[  20   41  120  174  197  211  226  229  231  258  294  297  317  406
  431  535  591  614  678  714  738  804  812  814  853  880 1024 1134
 1191 1196 1226 1263 1273 1291 1298 1443 1603 1630], shape=(38,), dtype=int32)
tf.Tensor([  30  473 1308], shape=(3,), dtype=int32)


Features on the node sets or edge sets of the merged graph have the set's total size as their first dimension, followed by the dimensions of individual feature values. This makes them compatible with many standard layers like tf.keras.layers.Dense, which accept an unknown batch size as the first dimension, except the "batch size" of a node feature is the total number of nodes in the batch.


In [ ]:
print(merged_graph.node_sets["paper"].total_size.numpy())
print(merged_graph.node_sets["paper"]["feat"].shape)

1811
(1811, 128)


**Please remember:** Each TF-GNN Model needs to call `.merge_batch_to_components()` at one point after the final input batches for each model replica have been formed but before the actual GNN model starts. For TPUs, this and the subsequent padding to fixed sizes has to happen before data is fed into the trained Model.

## Model building and training

We use TensorFlow's [Distribution Strategy](https://www.tensorflow.org/guide/distributed_training) API to write a model that can train in parallel on multiple [Cloud TPUs](https://cloud.google.com/tpu), multiple GPUs, or maybe just locally on CPU. (This is needed on Colab to use Cloud TPUs. This is not required to use the single GPU on a Colab, but we might as well show how it's done for the general case.)

In [ ]:
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
  print("Running on TPU ", tpu_resolver.cluster_spec().as_dict()["worker"])
except ValueError:
  tpu_resolver = None
if tpu_resolver:
  print("Using TPUStrategy")
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  assert isinstance(strategy, tf.distribute.TPUStrategy)
elif tf.config.list_physical_devices("GPU"):
  gpu_list = !nvidia-smi -L
  print("\n".join(gpu_list))
  print(f"Using MirroredStrategy for GPUs")
  strategy = tf.distribute.MirroredStrategy()
else:
  strategy = tf.distribute.get_strategy()
  print(f"Using default strategy")
print(f"Found {strategy.num_replicas_in_sync} replicas in sync")

GPU 0: Tesla T4 (UUID: GPU-fc6155fe-adbf-f710-3e1d-b4f981ab8cde)
Using MirroredStrategy for GPUs
Found 1 replicas in sync


### Padding (for TPUs)


Training on Cloud TPUs involves just-in-time compilation of a TensorFlow model to TPU code, and requires fixed shapes for all Tensors involved. To achieve that for graph data with variable numbers of nodes and edges, we need to pad each input Tensor to some fixed maximum size. For training on GPUs or CPU, this extra step is not necessary.

In [ ]:
#@title Pad batched GraphTensors to fixed sizes?
#@markdown By default (`None`), padding is used for TPUs only.
use_padding = None #@param ["None", "True", "False"] {type:"raw"}
if use_padding is None:
  use_padding = isinstance(strategy, tf.distribute.TPUStrategy)
print("Padding is", ["OFF", "ON"][use_padding])
if isinstance(strategy, tf.distribute.TPUStrategy) and not use_padding:
  raise ValueError("Padding is required for running on TPU")

Padding is OFF


For the validation dataset, we need to make sure that every batch of examples fits within the fixed sizes, no matter how the parallelism in the input pipeline ends up combining examples into batches. Therefore, we use a rather generous estimate, basically scaling each Tensor's observed maximum size by a factor of batch_size. If that were to run into limitations of accelerator memory, we'd rather shrink the batch size than lose examples.

The dataset in this example is not too big, so we can scan it within a few minutes to determine constraints large enough for all inputs. (For huge datasets under your control, it may be worth inferring an upper bound from the sampling spec instead.)

In [ ]:
global_batch_size = 128
validation_global_batch_size = global_batch_size
assert validation_global_batch_size % strategy.num_replicas_in_sync == 0, "divisibility required"
validation_replica_batch_size = validation_global_batch_size // strategy.num_replicas_in_sync
print(f"Validation uses a global batch size of {validation_global_batch_size} "
      f"({validation_replica_batch_size} per replica).")
validation_size_constraints = None
if use_padding:
  # The "paper" node set needs at least one node in each graph component,
  # incl. those added for padding, because the model will read out the state
  # of the sampled subgraph's root node from each component.
  min_nodes_per_component = {"paper": 1}
  validation_size_constraints = tfgnn.find_tight_size_constraints(
      _get_dataset(input_file_pattern, shuffle=False,
                  filter_fn=_is_in_split("validation"),  # For OGB only.
      ).map(parse),
      target_batch_size=validation_replica_batch_size,
      min_nodes_per_component=min_nodes_per_component)
  print(f"Validation data is padded to: {validation_size_constraints}")

Validation uses a global batch size of 32 (32 per replica).


For the training dataset, TF-GNN allows you to optimize more aggressively for large batch sizes: size constraints satisfied by 100% of all possible inputs would have to accommodate the rare combination of many large examples in one batch.

Instead, we use size constraints that will fit *close to* 100% of the randomly drawn training batches. This is not covered by the theory supporting stochastic gradient descent (which calls for examples drawn independently at random), but in practice, it often works, and allows larger batch sizes within the limits of accelerator memory, and hence faster convergence of the training. Running the code block below with `use_padding=True` shows the sizes required for some success ratios. (Compare them to the constraints above, which are satisfied 100% at a fourth of the batch size.)


In [ ]:
training_global_batch_size = global_batch_size
assert training_global_batch_size % strategy.num_replicas_in_sync == 0, "divisibility required"
training_replica_batch_size = training_global_batch_size // strategy.num_replicas_in_sync
print(f"Training uses a batch size of {training_global_batch_size} "
      f"({training_replica_batch_size} per replica).")
training_size_constraints = None
if use_padding:
  success_ratios = [0.90, 0.98, 0.99]
  constraints = tfgnn.learn_fit_or_skip_size_constraints(
      _get_dataset(input_file_pattern,
                   filter_fn=_is_in_split("train"),  # For OGB only.
      ).map(parse),
      training_replica_batch_size,
      min_nodes_per_component=min_nodes_per_component,
      success_ratio=success_ratios, sample_size=20000)
  for sr_idx, sr in enumerate(success_ratios):
    print(f"Success ratio {sr} requires: {constraints[sr_idx]}")
  sr_idx = 2
  training_size_constraints = constraints[sr_idx]
  print(f"\nSelected success ratio: {success_ratios[sr_idx]}.")
  print(f"Training data is padded to: {training_size_constraints}")

Training uses a batch size of 128 (128 per replica).


### Preprocessing the input features

As usual in TensorFlow, the non-trainable transformations of the input features are split off into a `Dataset.map()` call while the model proper consists of the trainable and accelerator-compatible parts. However, even this non-trainable part is put into a Keras model, which is a convenient way to track resources for export (such as lookup tables). The following code cells provide a fully worked example; for more background information, please see our [Input pipeline](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/input_pipeline.md) guide.

In [ ]:
# Callbacks used by tfgnn.keras.layers.MapFeatures below.

def _preprocess_node_features(node_set, *, node_set_name):
  if node_set_name == "paper":
    return {
        # Retain the word2vec embedding unchanged.
        "feat": node_set["feat"],
        # Keep the label, until popped later on.
        "labels": node_set["labels"]}
  elif node_set_name == "author":
    # There are no useful features. Instead, we create a tensor of hidden node
    # states that are empty, i.e., with shape [batch_size, (num_nodes), 0].
    return {"empty_state": tfgnn.keras.layers.MakeEmptyFeature()(node_set)}
  elif node_set_name == "field_of_study":
    # Convert the string id to an index into an embedding table.
    # Conveniently, this Keras layer can handle RaggedTensors.
    return {"hashed_id": tf.keras.layers.Hashing(num_bins=50_000)(
        node_set["#id"])}
  elif node_set_name == "institution":
    # Convert the string id to an index into an embedding table.
    return {"hashed_id": tf.keras.layers.Hashing(num_bins=6_500)(
        node_set["#id"])}
  else:
    raise KeyError(f"Unexpected node_set_name='{node_set_name}'")

def _drop_all_features(graph_piece, **unused_kwargs):
  return {}

In [ ]:
def _make_preprocessing_model(graph_tensor_spec, size_constraints):
  """Returns Keras model to preprocess a batched and parsed GraphTensor."""
  graph = input_graph = tf.keras.layers.Input(type_spec=graph_tensor_spec)

  # Convert input features to suitable representations for use on GPU/TPU.
  # Drop unused features (like id strings for tracking the source of examples).
  graph = tfgnn.keras.layers.MapFeatures(
      node_sets_fn=_preprocess_node_features,
      edge_sets_fn=_drop_all_features,
      context_fn=_drop_all_features)(graph)
  assert "labels" in graph.node_sets["paper"].features

  ### IMPORTANT: All TF-GNN modeling code assumes a GraphTensor of shape []
  ### in which the graphs of the input batch have been merged to components of
  ### one contiguously indexed graph. There are no edges between components,
  ### so no information flows between them.
  graph = graph.merge_batch_to_components()

  # Optionally, pad to size_constraints (required for TPU).
  if size_constraints:
    graph, mask = tfgnn.keras.layers.PadToTotalSizes(size_constraints)(graph)
  else:
    mask = None

  # Split the label off the padded input graph.
  root_label = tfgnn.keras.layers.ReadoutFirstNode(
      node_set_name="paper", feature_name="labels")(graph)
  graph = graph.remove_features(node_sets={"paper": ["labels"]})
  assert "labels" not in graph.node_sets["paper"].features

  outputs = (graph, root_label) if mask is None else (graph, root_label, mask)
  return tf.keras.Model(input_graph, outputs)

The preprocessing model merges its batch of input graphs into one contiguously indexed graph with multiple components (as explained above), and then pads it (if applicable). After that, the dataset can no longer be rebatched to split it up between replicas, so we use `tf.distribute.Strategy.distribute_datasets_from_function()` to build a `DistributedDataset` of per-replica inputs. (If it weren't for TPUs or distribution strategies, the call to `merge_batch_to_components()` could be deferred to the start of the trained model, and we could leave it to `Model.fit()` to split up a single Dataset.)

In [ ]:
example_input_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)

def _get_preprocessed_dataset(
    input_context, split_name, per_replica_batch_size, size_constraints):
  training = split_name == "train"
  ds = _get_dataset(input_file_pattern, shuffle=training,
                    filter_fn=_is_in_split(split_name),  # For OGB only.
                    input_context=input_context)
  if training:
    ds = ds.repeat()
  # There is no need to drop_remainder when batching, even for TPU:
  # padding the GraphTensor can handle variable numbers of components.
  ds = ds.batch(per_replica_batch_size)
  ds = ds.map(tfgnn.keras.layers.ParseExample(example_input_spec))
  if training and size_constraints:
    ds = ds.filter(functools.partial(tfgnn.satisfies_size_constraints,
                                     total_sizes=size_constraints))
  ds = ds.map(_make_preprocessing_model(ds.element_spec, size_constraints),
              deterministic=False, num_parallel_calls=tf.data.AUTOTUNE)
  return ds

def _get_distributed_preprocessed_dataset(
    strategy, split_name, per_replica_batch_size, size_constraints):
  """Returns DistributedDataset and its per-replica element_spec."""
  return strategy.distribute_datasets_from_function(functools.partial(
      _get_preprocessed_dataset,
      split_name=split_name, per_replica_batch_size=per_replica_batch_size,
      size_constraints=size_constraints))

train_ds = _get_distributed_preprocessed_dataset(
    strategy, "train",
    training_replica_batch_size, training_size_constraints)
valid_ds = _get_distributed_preprocessed_dataset(
    strategy, "validation",
    validation_replica_batch_size, validation_size_constraints)

### The GNN core of the model



We are going to use the same Model object for training, validation, and export for inference, so we need to build it from an input type spec with generic tensor shapes. (For TPUs, using it on a *dataset* with fixed-size elements will suffice.) Rather than spelling out the type spec by hand, we create a non-distributed, non-padded dummy dataset whose element spec reflects the preprocessing. (That's cheap, as long as we don't create an iterator on it.)

In [ ]:
build_model_graph_tensor_spec, *_ = _get_preprocessed_dataset(
    input_context=None, split_name="train",
    per_replica_batch_size=2, size_constraints=None).element_spec

In [ ]:
def _build_model(
    # To be called with the build_model_graph_tensor_spec from above.
    graph_tensor_spec,
    # Dimensions of initial states.
    field_of_study_dim=32,
    institution_dim=16,
    paper_dim=512,
    # Dimensions for message passing.
    message_dim=128,
    next_state_dim=128,
    # Dimension for the logits.
    num_classes=349,
    # Other hyperparameters.
    l2_regularization=6e-6,
    dropout_rate=0.2,
    use_layer_normalization=True,
):
  # Model building with Keras's Functional API starts with an input object
  # (a placeholder for future inputs). This works for composite tensors, too.
  graph = input_graph = tf.keras.layers.Input(type_spec=graph_tensor_spec)

  # The initial hidden states for each node set.
  def set_initial_node_state(node_set, *, node_set_name):
    if node_set_name == "paper":
      # A trainable transformation of the word2vec input features.
      return tf.keras.layers.Dense(paper_dim)(node_set["feat"])
    elif node_set_name == "author":
      # The empty initial state for each node was created in preprocessing
      # and now comes out here with the correct shape (fixed in case of TPU).
      return node_set["empty_state"]
    elif node_set_name == "field_of_study":
      # A trainable embedding (as discussed above).
      return tf.keras.layers.Embedding(50_000, field_of_study_dim)(
          node_set["hashed_id"])
    elif node_set_name == "institution":
      # A trainable embedding (as discussed above).
      return tf.keras.layers.Embedding(6_500, institution_dim)(
          node_set["hashed_id"])
    else:
      raise KeyError(f"Unexpected node_set_name='{node_set_name}'")
  graph = tfgnn.keras.layers.MapFeatures(
      node_sets_fn=set_initial_node_state, name="init_states")(graph)

  # Abbreviations for repeated building blocks in the GNN.
  def dense(units, *, use_layer_normalization=False):
    """A Dense layer with regularization (L2 and Dropout) and normalization."""
    regularizer = tf.keras.regularizers.l2(l2_regularization)
    result = tf.keras.Sequential([
        tf.keras.layers.Dense(
            units,
            activation="relu",
            use_bias=True,
            kernel_regularizer=regularizer,
            bias_regularizer=regularizer),
        tf.keras.layers.Dropout(dropout_rate)])
    if use_layer_normalization:
      result.add(tf.keras.layers.LayerNormalization())
    return result

  def convolution(message_dim, receiver_tag):
    return tfgnn.keras.layers.SimpleConv(dense(message_dim), "sum",
                                         receiver_tag=receiver_tag)

  def next_state(next_state_dim, use_layer_normalization):
    return tfgnn.keras.layers.NextStateFromConcat(dense(next_state_dim, use_layer_normalization=use_layer_normalization))

  # The GNN "core" of the model.
  # Convolutions let data flow towards the specified endpoint of edges, e.g.,
  # along "cites" edges from TARGET (cited paper) to SOURCE (citing paper).
  # See the text below the colab cell for more explanations.
  for i in range(4):
    graph = tfgnn.keras.layers.GraphUpdate(node_sets={
        "paper": tfgnn.keras.layers.NodeSetUpdate(
            {"cites": convolution(message_dim, tfgnn.SOURCE),
             "written": convolution(message_dim, tfgnn.SOURCE),
             "has_topic": convolution(message_dim, tfgnn.SOURCE)},
            next_state(next_state_dim, use_layer_normalization)),
         "author": tfgnn.keras.layers.NodeSetUpdate(
            {"writes": convolution(message_dim, tfgnn.SOURCE),
             "affiliated_with": convolution(message_dim, tfgnn.SOURCE)},
            next_state(next_state_dim, use_layer_normalization)),
         })(graph)

  # Read out the hidden state of the root node of each **component** in the
  # graph (cf. .merge_batch_to_components() above).
  root_states = tfgnn.keras.layers.ReadoutFirstNode(node_set_name="paper")(graph)
  # Put a linear classifier on top. (Never use dropout here.)
  logits = tf.keras.layers.Dense(num_classes)(root_states)

  return tf.keras.Model(input_graph, logits)

At its core, the GNN model above consists of multiple rounds of graph updates, each expressed in Keras as

```
graph = GraphUpdate(...)(graph)
```

with a `graph` of type `GraphTensor`.

Several of the [tensorflow_gnn/models](https://github.com/tensorflow/gnn/tree/main/tensorflow_gnn/models) provide ready-to-use `GraphUpdate` classes. The basic "[OGBN-MAG end-to-end](https://colab.research.google.com/github/tensorflow/gnn/blob/master/examples/notebooks/ogbn_mag_e2e.ipynb)" tutorial demonstrates the use of the [VanillaMPNNGraphUpdate](https://github.com/tensorflow/gnn/tree/main/tensorflow_gnn/models/vanilla_mpnn). In this advanced tutorial, we dig deeper and explicitly describe how such a graph update can be built with the generic `tfgnn.keras.layers.GraphUpdate` class.

In the provided example, we perform four rounds of graph updates. A graph update consists of updates to node sets, that is to say, replacing the `"hidden_state"` (` == tfgnn.HIDDEN_STATE`) feature on each of them. The node set updates of one graph update happen in parallel, that is, on the same input `graph`.

Each node set update is expressed as a `NodeSetUpdate` layer, which receives the input `graph` and returns a new hidden state for the node set it gets applied to. The new hidden state is computed with the given next-state layer from the node set's prior state and the aggregated results from each incoming edge set.

For example, each round of the model above computes a new state for node set "paper" by applying `dense(next_state_dim)` to the concatenation of

  * the prior state `graph.node_sets["paper"][tfgnn.HIDDEN_STATE]`,
  * the result of `convolution(message_dim)(graph, edge_set_name="cites")`,
  * the result of `convolution(message_dim)(graph, edge_set_name="written")` and
  * the result of `convolution(message_dim)(graph, edge_set_name="has_topic")`.

A convolution on an edge set computes a value for each edge (a "message") as a trainable function of the node states at both endpoints, and then aggregates the results at the receiver nodes by forming the sum (or mean or max) over all incoming edges.

For example, the convolution on edge set "written" concatenates the node state of each edge's incident "paper" and "author" node, applies `dense(message_dim)`, and sums the results over the edges incident to each "paper" node (that is, at the `SOURCE` node of each edge).

Notice that the conventional names *source* and *target* for the endpoints of a directed edge do **not** prescribe the direction of information flow: each "written" edge logically goes from a paper to its author (so the "author" node is its `TARGET`), yet this model lets the data flow towards the paper (and the "paper" node is its `SOURCE`). In fact, sampled subgraphs have edges directed away from the root node, so data flow towards the root often goes from `TARGET` to `SOURCE`.

> Note on terminology: Convolutions are best known in deep learning for convolutional neural networks on image data, in which they aggregate information from a small, fixed, implicitly understood neighborhood of each element in a pixel grid. The term loosely carries over to graphs by interpreting edges as explicit, variable definitions of a node's neighborhood.

The code above creates fresh Convolution and NextState layer objects for each edge set and node set, resp., and for each round of updates. This means they all have separate trainable weights. If desired, weight sharing is possible in the standard Keras way by sharing convolution and next-state layer objects, provided the input sizes match.

For more information on defining your own GNN models (including those with edge and context states), please refer to the [TF-GNN Modeling Guide](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/gnn_modeling.md).


### Training the model

To train the Keras Model, as usual, we build it under the distribution strategy scope, compile and fit it.


In [ ]:
with strategy.scope():
  model = _build_model(build_model_graph_tensor_spec)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = [tf.keras.metrics.SparseCategoricalAccuracy(),
             tf.keras.metrics.SparseCategoricalCrossentropy(from_logits=True)]

**IMPORTANT:** In `model.compile()`, pass `weighted_metrics=[...]` instead of plain `metrics=[...]` if your code ever gets used with GraphTensors that are padded to fixed sizes. Otherwise the padding mask is not applied in metric computations, so that padding values interfere with the metrics (making them worse than they really are).

Training for 5 epochs of sampled subgraphs takes a few hours on a colab with one GPU and should achieve an accuracy above 0.47. Lucky runs with many more epochs can reach 0.51. (Training with a Cloud TPU runtime is faster, but note that this set-up is not optimized specifically for TPUs.)

To keep this demo more interactive, we let Keras train and evaluate on fractions of a true epoch for a few minutes only. Of course the resulting accuracy will be poor. To fix that, feel free to edit the `epoch_divisor` according to your patience and ambition. ;-)

In [ ]:
epoch_divisor = 100  # To speed up the interactive demo_ds
steps_per_epoch = num_training_samples // global_batch_size // epoch_divisor
validation_steps = num_validation_samples // global_batch_size  // epoch_divisor
epochs = 5
learning_rate = tf.keras.optimizers.schedules.CosineDecay(0.001, steps_per_epoch*epochs)
with strategy.scope():
  model.compile(tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=loss, weighted_metrics=metrics, steps_per_execution=20)

  history = model.fit(
      train_ds,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_data=valid_ds,
      validation_steps=validation_steps,
  )

Epoch 1/5


49/49 [==============================] - 74s 2s/step - loss: 5.2805 - sparse_categorical_accuracy: 0.0611 - sparse_categorical_crossentropy: 5.2522 - val_loss: 5.1825 - val_sparse_categorical_accuracy: 0.0547 - val_sparse_categorical_crossentropy: 5.1551
Epoch 2/5
49/49 [==============================] - 35s 706ms/step - loss: 4.9080 - sparse_categorical_accuracy: 0.0834 - sparse_categorical_crossentropy: 4.8812 - val_loss: 4.8607 - val_sparse_categorical_accuracy: 0.0734 - val_sparse_categorical_crossentropy: 4.8343
Epoch 3/5
49/49 [==============================] - 30s 612ms/step - loss: 4.6028 - sparse_categorical_accuracy: 0.1258 - sparse_categorical_crossentropy: 4.5766 - val_loss: 4.5573 - val_sparse_categorical_accuracy: 0.1016 - val_sparse_categorical_crossentropy: 4.5313
Epoch 4/5
49/49 [==============================] - 35s 710ms/step - loss: 4.4082 - sparse_categorical_accuracy: 0.1432 - sparse_categorical_crossentropy: 4.3822 - val_loss: 4.4527 - val_sparse_categorical_accu

## Export for inference

At the end of training, a SavedModel is exported for inference. C++ inference environments like TF Serving do not support input of extension types like GraphTensor, so we export the model with a SavedModel Signature that accepts a batch of serialized tf.Examples and preprocesses them like training did.

In [ ]:
serving_input = tf.keras.layers.Input(shape=[], dtype=tf.string, name="examples")
preproc_input = tfgnn.keras.layers.ParseExample(example_input_spec)(serving_input)
preproc_model = _make_preprocessing_model(preproc_input.type_spec,
                                          size_constraints=None)
model_input, _ = preproc_model(preproc_input)  # Drop labels. (No mask.)
logits = model(model_input)
serving_output = {
    # SavedModel signature outputs are keyed by the name of the last layer.
    # Restored Keras model outputs preserve the dict seen here.
    # This code puts the same key into both places.
    "logits": tf.keras.layers.Layer(name="logits")(logits),
    "probabilities": tf.keras.layers.Layer(name="probabilities")(
        tf.math.softmax(logits))}
serving_model = tf.keras.Model(serving_input, serving_output)

In [ ]:
export_path = "/tmp/exported_keras_model"
!rm -r {export_path}
# Save everything on the Colab host (even the variables from TPU memory).
save_options = tf.saved_model.SaveOptions(experimental_io_device="/job:localhost")
serving_model.save(export_path, include_optimizer=False, options=save_options)

rm: cannot remove '/tmp/exported_keras_model': No such file or directory


/usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:522: UserWarning: Encoding a StructuredValue with type tensorflow_gnn.GraphTensorSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:522: UserWarning: Encoding a StructuredValue with type tensorflow_gnn.ContextSpec.v2; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:522: UserWarning: Encoding a StructuredValue with type tensorflow_gnn.NodeSetSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; 

Note that any resources for preprocessing (like lookup tables) have to be attached to the saved object. Our current code has none yet, but composing Keras models as above will take care of it.



For demonstration, let's call the exported model on the example dataset from above, but without labels. We load it as a SavedModel, like TF Serving would. (Using `tf.keras.models.load_model()` instead would rebuild the original Keras layers; see TensorFlow's [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load) tutorial for more.)

Note: After connecting this Colab to a TPU worker, explicit device placements are necessary to do the test on the colab host (which has the `/tmp` directory). You can omit those when loading the SavedModel elsewhere.

In [ ]:
with tf.device("/job:localhost"):
  restored_model = tf.saved_model.load(export_path)

In [ ]:
def _clean_example_for_serving(serialized):
  example = tf.train.Example.FromString(serialized)
  example.features.feature.pop("nodes/paper.labels")
  return example.SerializeToString()

num_examples = 10
clean_examples = [_clean_example_for_serving(gt.numpy()) for gt in itertools.islice(demo_ds, num_examples)]

with tf.device("/job:localhost"):
  clean_ds = tf.data.Dataset.from_tensor_slices(clean_examples)
  for serialized_example in clean_ds.batch(num_examples).take(1):
    outputs = restored_model.signatures["serving_default"](
        examples=serialized_example)
    probabilities = outputs["probabilities"].numpy()
    classes = probabilities.argmax(axis=1)
    for i, c in enumerate(classes):
      print(f"The predicted class for input {i} is {c:3} "
            f"with predicted probability {probabilities[i, c]:.4}")

The predicted class for input 0 is 258 with predicted probability 0.1439
The predicted class for input 1 is   1 with predicted probability 0.3566
The predicted class for input 2 is   1 with predicted probability 0.3649
The predicted class for input 3 is 134 with predicted probability 0.1926
The predicted class for input 4 is 258 with predicted probability 0.1489
The predicted class for input 5 is   1 with predicted probability 0.2027
The predicted class for input 6 is 134 with predicted probability 0.1512
The predicted class for input 7 is 134 with predicted probability 0.0338
The predicted class for input 8 is 258 with predicted probability 0.05497
The predicted class for input 9 is 134 with predicted probability 0.03303


Recall that this is not a fully trained model, so the results will be inaccurate, unless you changed `epoch_divisor` above.

## Next steps

This tutorial has shown how to solve a node classification problem in a large graph with TF-GNN, using
  * the graph sampler tool to obtain manageable inputs for each classification target,
  * a TensorFlow model built in Colab with tfgnn.keras.layers.

The [Data Preparation and Sampling](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/data_prep.md) guide describes how you can create training data for other datasets.

The [Modeling](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/gnn_modeling.md) guide explains how to use other GNN architectures or write your own.